In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests

from datetime import timedelta, date
import itertools

import re
import folium
%matplotlib inline

In [2]:
#2015 m. Lietuvos Respublikos savivaldybių tarybų rinkimai
url2015 = 'https://www.vrk.lt/statiniai/puslapiai/2015_savivaldybiu_tarybu_rinkimai/output_lt/rezultatai_daugiamand_apygardose/rezultatai_daugiamand_apygardose1turas.html'

result2015 = requests.get(url2015)
c2015 = result2015.content
soup2015 = BeautifulSoup(c2015,'lxml')
table2015 = soup2015.find('table', style="margin-left:0px;margin-right:0px;width:90%")
links2015 = table2015.find_all('a', href=True)

url = [] #Nuorodos į kiekvienos savivaldybės rezultatus
apyg = [] #Savivaldybių pavadinimai

for link in links2015:
    url.append('https://www.vrk.lt/statiniai/puslapiai/2015_savivaldybiu_tarybu_rinkimai/output_lt/rezultatai_daugiamand_apygardose/apygardos_rezultatai_apylinkese'+link['href'][-9:])
    apyg.append(link.contents[0].strip().replace('  ', ' '))
    

data2015=[] #Balsavimo rezultatai savivaldybėse, apylinkių skaičius, dalyvavusių skaičius, galiojantys, negaliojantys biuletenys
rows2015 = table2015.find_all('tr')[2:-1]

for row2015 in rows2015:
    cols2015 = row2015.find_all('td')
    for col2015 in cols2015:
        text2015=col2015.find(text=True)
        data2015.append(text2015)
df2015 = pd.DataFrame(np.array(data2015).reshape(int(len(data2015)/13),13))
df2015[0]=apyg
df2015 = df2015.drop([2, 3, 4, 7], axis=1)
df2015.columns = ['Apyg', 'ApylSk', 'RinkSk', 'Dalyv', 'NegalMer', 'NegalTar', 'GalMer', 'GalTar', 'PirmNuoGal']

In [3]:
apyg_dfs0 = []
ap_dfs0=[]

for apyg_url, apyg_apyg in zip(url, apyg):

    apyg_result = requests.get(apyg_url)
    apyg_c = apyg_result.content
    apyg_soup = BeautifulSoup(apyg_c,'lxml')
    apyg_table = apyg_soup.find_all('table', style="width:80%", border="0")
    apyg_table = apyg_table[0]
    apyg_links = apyg_table.find_all('a', href=True)

    url_apyg = [] #sarasas nuorodu apylinkes rezultatu
    apyl = [] #sarasas apylinkiu


    for link in apyg_links:
        url_apyg.append('https://www.vrk.lt/statiniai/puslapiai/2015_savivaldybiu_tarybu_rinkimai/output_lt/rezultatai_daugiamand_apygardose/'+link['href'])
        apyl.append(link.contents[0].strip().replace('  ', ' '))

###############    
        
    for ap_url, ap_name in zip(url_apyg, apyl):
        ap_result = requests.get(ap_url)
        ap_c = ap_result.content
        ap_soup = BeautifulSoup(ap_c,'lxml')
        ap_table = ap_soup.find('table', style="width:75%", border="0")
        ap_rows = ap_table.find_all('tr')[2:-1]
        ap_data=[]
        for ap_row in ap_rows:
            ap_cols = ap_row.find_all('td')
            for ap_col in ap_cols:
                ap_text=ap_col.find(text=True)
                ap_data.append(ap_text)
        ap_df = pd.DataFrame(np.array(ap_data).reshape(int(len(ap_data)/9),9))
        ap_df['apylinke']=ap_name
        ap_df['apygarda']=apyg_apyg
        ap_dfs0.append(ap_df)
   
    
###############    
    apyg_rows = apyg_table.find_all('tr')[2:-1]
    apyg_data=[]

    for apyg_row in apyg_rows:
        apyg_cols = apyg_row.find_all('td')
        for apyg_col in apyg_cols:
            apyg_text=apyg_col.find(text=True)
            apyg_data.append(apyg_text)
    apyg_df = pd.DataFrame(np.array(apyg_data).reshape(int(len(apyg_data)/10),10))
    apyg_df[0]=apyl
    apyg_df = apyg_df.drop([8, 9], axis=1)
    apyg_df['apygarda'] = apyg_apyg
    apyg_dfs0.append(apyg_df)
apyg_dfs = pd.concat(apyg_dfs0, ignore_index=True)
apyg_dfs.columns = ['apylinke', 'rink_sk', 'dalyvavo', 'dalyvavo_proc', 'negalioja', 'negalioja_proc', 'galioja', 'galioja_proc', 'apygarda']
apyg_dfs[['rink_sk', 'dalyvavo', 'negalioja', 'galioja']] = apyg_dfs[['rink_sk', 'dalyvavo', 'negalioja', 'galioja']].astype(int)
apyg_dfs=apyg_dfs.replace(',', '.', regex=True)
apyg_dfs[['dalyvavo_proc', 'negalioja_proc', 'galioja_proc']] = apyg_dfs[['dalyvavo_proc', 'negalioja_proc', 'galioja_proc']].astype(float)

###############
ap_dfs = pd.concat(ap_dfs0, ignore_index=True)
ap_dfs = ap_dfs.drop([2, 6, 8], axis=1)
ap_dfs.columns = ['nr', 'kandidatas', 'apylinkeje', 'pastu','viso', 'su_pirmumo', 'apylinke', 'apygarda']
ap_dfs[['apylinkeje', 'pastu', 'viso', 'su_pirmumo']] = ap_dfs[['apylinkeje', 'pastu', 'viso', 'su_pirmumo']].astype(int)   

In [5]:
rinkimai = '2015sav'
ap_dfs['rinkimai'] = rinkimai
ap_dfs.to_csv(rinkimai+'.csv', index=False, header=True, encoding="cp1257")